In [1]:
#!pip install -i https://test.pypi.org/simple/ nsc-test

# Setting up

In [2]:
import nsc
import numpy as np
from nsc.distributions import MultivariateCoupledNormal
from nsc.math.entropy import importance_sampling_integrator
from nsc.math.entropy_norm import coupled_normal_entropy, biased_coupled_probability_norm, coupled_probability_norm, coupled_cross_entropy_norm, coupled_entropy_norm, coupled_kl_divergence_norm

Importing NSC lib v0.0.4.12.


In [3]:
# Set the locs.
loc = np.array([[0, 0,]])
# Set the scales.
scale = np.array([[1, 1]])

# Find the number of batches.
n_batches = loc.shape[0]

# Set the kappa.
kappa = 0.0

# Initialize a MultivariateCoupledNormal object.
mvn = MultivariateCoupledNormal(loc=loc, scale=scale, kappa=kappa)

In [4]:
[1], []

([1], [])

In [5]:
mvn._norm_term

array([6.28318531])

In [6]:
mvn._alpha

2

In [7]:
mvn._sigma

array([[[1., 0.],
        [0., 1.]]])

# Testing `sample_n`

In [8]:
# Set the number of samples per batched distribution.
n = 10000
# Sample n observations from each batched distribution.
samples = mvn.sample_n(n)

print(f"Expected dimensions of samples: {loc.shape, n, loc.shape}")
print(f"Actual dimensions of samples: {samples.shape}")

Expected dimensions of samples: ((1, 2), 10000, (1, 2))
Actual dimensions of samples: (1, 10000, 2)


In [9]:
for i in range(n_batches):
    print(f"Mean of distribution 1: {tuple(loc[i])}")
    print(f"Mean of samples from distribution 1: {tuple(samples[i].mean(axis=0))}")
    print(f"Std. Dev. of distribution 1: {tuple(scale[i])}")
    print(f"Std. Dev. of samples from distribution 1: {tuple(samples[i].std(axis=0))}")
    print("\n")

Mean of distribution 1: (0, 0)
Mean of samples from distribution 1: (0.004302784844360591, 0.003907214502454675)
Std. Dev. of distribution 1: (1, 1)
Std. Dev. of samples from distribution 1: (1.001491905168859, 0.9977306632997097)




# Testing `prob` with no batches (single distribution)

In [10]:
from scipy.stats import multivariate_normal

# Set the locs.
loc_1a = np.array([loc[0]])
# Set the scales.
scale_1a =  np.array([scale[0]])

# Initialize a MultivariateCoupledNormal object.
mvn_1a = MultivariateCoupledNormal(loc=loc_1a, scale=scale_1a, kappa=kappa)
# Initialize a scipy multivariate_normal object.
mvn_scipy_1a = multivariate_normal(loc_1a[0], scale_1a[0]**2)

# Set the number of samples.
n = 100
# Sample n observations from the distribution.
samples_1a = mvn_1a.sample_n(n)

# Get the probability densities from MVN object.
mvn_probs_1a = mvn_1a.prob(samples_1a)
print(f"Shape of samples: {mvn_probs_1a.shape}")
# Reshape them to match the scipy's distribution output.
mvn_probs_1a = mvn_1a.prob(samples_1a).reshape(-1,)

# Get the probability densities from scipy object.
scipy_probs_1a = mvn_scipy_1a.pdf(samples_1a)

print(f"The densities from the Coupled MVN and SciPy MVN are close: {np.allclose(mvn_probs_1a, scipy_probs_1a)}")

Shape of samples: (1, 100, 1, 1)
The densities from the Coupled MVN and SciPy MVN are close: True


# Testing `prob` with batches (multiple distributions)

In [11]:
mvn_probs = mvn.prob(samples)

# Loop through the 
for i in range(n_batches):
    # Get the i-th distributions sample densities and reshape them to be 1-D.
    dist_1_probs = mvn_probs[i].reshape(-1,)
    # Initialize a scipy multivariate_normal object.
    mvn_scipy_1 = multivariate_normal(loc[i], scale[i]**2)
    # Get the probability densities from scipy object.
    scipy_probs_1 = mvn_scipy_1.pdf(samples[i])
    
    print(f"The densities from the Coupled MVN and SciPy MVN are close: {np.allclose(dist_1_probs, scipy_probs_1)}")
    print("\n")

The densities from the Coupled MVN and SciPy MVN are close: True




C:\Users\jkcle\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# `importance_sampling_integrator`

In [12]:
importance_sampling_integrator(mvn.prob, mvn.prob, mvn.sample_n, n=10000, seed=1)

C:\Users\jkcle\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[[1.]]])

In [13]:
importance_sampling_integrator(mvn.prob, mvn_1a.prob, mvn.sample_n, n=10000, seed=1)

C:\Users\jkcle\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[[1.]]])

# `coupled_normal_entropy`

In [14]:
#coupled_normal_entropy(mvn._sigma, kappa=0.0)

C:\Users\jkcle\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
#coupled_normal_entropy(mvn._sigma, kappa=0.003)

In [16]:
#mvn._sigma

In [17]:
0.5 * np.log((2*np.pi*np.exp(1)) ** mvn.loc[0].shape[0] * np.linalg.det(mvn._sigma[0]))

2.8378770664093453

# `biased_coupled_probability_norm`

In [18]:
new_mvn = biased_coupled_probability_norm(mvn, kappa=0.1, alpha=2)

print(new_mvn.loc)
print(new_mvn._sigma)
print(new_mvn.kappa)

[[0 0]]
[[[0.85714286 0.        ]
  [0.         0.85714286]]]
0.0


In [19]:
new_mvn_samples = new_mvn.sample_n(100)

# `coupled_probability_norm`

In [20]:
new_mvn_pdf = coupled_probability_norm(mvn, kappa=0.1, alpha=2.0)

np.all(new_mvn_pdf(new_mvn_samples) == new_mvn.prob(new_mvn_samples))

True

# `coupled_cross_entropy_norm`

In [21]:
coupled_cross_entropy_norm(
    mvn,
    new_mvn,
    kappa=0.0, 
    alpha=2.0, 
    root=False,
    n=10000,
    seed=1
)

C:\Users\jkcle\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[[2.85047335]]])

# `coupled_entropy_norm`

In [22]:
coupled_entropy_norm(
    mvn,
    kappa=0.0, 
    alpha=2.0, 
    root=False,
    n=10000,
    seed=1
)

C:\Users\jkcle\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[[2.83794589]]])

In [23]:
true_entropies = [
    0.5 * np.log((2*np.pi*np.exp(1)) ** mvn.loc[i].shape[0] * np.linalg.det(mvn._sigma[i])) for i in range(mvn._batch_shape[0])
]
true_entropies = np.array(true_entropies)
true_entropies.reshape(mvn._batch_shape[0], 1, 1)

C:\Users\jkcle\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[[2.83787707]]])

# `coupled_kl_divergence_norm`

In [24]:
coupled_kl_divergence_norm(
    mvn, 
    new_mvn, 
    kappa=0.0, 
    alpha=2.0, 
    root=False,
    n=10000,
    seed=1
)

array([[[0.01252746]]])

In [25]:
coupled_kl_divergence_norm(
    mvn, 
    mvn, 
    kappa=0.0, 
    alpha=2.0, 
    root=False,
    n=10000,
    seed=1
)

C:\Users\jkcle\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[[0.]]])

In [26]:
coupled_kl_divergence_norm(
    new_mvn, 
    new_mvn, 
    kappa=0.0, 
    alpha=2.0, 
    root=False,
    n=10000,
    seed=1
)

C:\Users\jkcle\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[[0.]]])

In [27]:
def kl_divergence_MVN(mu_1, mu_2, Sigma_1, Sigma_2):
    """
    This function calculates the KL divergence between 2 multivariate normal distributions.
    
    Inputs
    -------
    mu_1 : numpy ndarray
        A 1-D array representing the mean vector of the first distribution.
    mu_2 : numpy ndarray
        A 1-D array representing the mean vector of the second distribution.
    Sigma_1 : numpy ndarray
        A 2-D square array representing the covariance matrix of the first distribution.
    Sigma_2 : numpy ndarray
        A 2-D square array representing the covariance matrix of the second distribution.
        
    Returns
    -------
    kl_div : float
        The KL Divergence between distributions 1 and 2.
    """
    
    # Validate input shapes.
    assert ((mu_1.shape == mu_2.shape)
            & (len(mu_1.shape) == 1)), "Mean vectors must have the same shape and be 1-D."
    assert ((Sigma_1.shape[0] == Sigma_1.shape[1])
            & (Sigma_1.shape == Sigma_2.shape)), "Sigma_1 and Sigma_2 must be square matrices with matching shapes."
    assert (mu_1.shape[0] == Sigma_1.shape[0]), "If the mean vectors are n by 1, the covariance matrices must be n by n."
    
    # Calculate the difference between the 2nd mean vector and the first.
    mean_diff = mu_2 - mu_1
    # Find the inverse of Sigma_2.
    Sigma_2_inv = np.linalg.inv(Sigma_2)
    
    # Get the dimensionality of the two distributions based on the first
    # dimension of the mu_1 vector.
    d = mu_1.shape[0]
    
    # Calculate the KL divergence.
    kl_div = np.log(np.linalg.det(Sigma_2)) - np.log(np.linalg.det(Sigma_1))
    kl_div -= d
    kl_div += np.trace(np.matmul(Sigma_2_inv, Sigma_1))
    kl_div += np.matmul(np.matmul(mean_diff.T, Sigma_2_inv), mean_diff)
    kl_div *= 0.5
    
    # Return the KL divergence.
    return kl_div

C:\Users\jkcle\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
kl_divergence_MVN(mvn.loc[0], new_mvn.loc[0], mvn.scale[0]**2, new_mvn.scale[0]**2)

0.012515986839408466

In [146]:
# Set the number of distributions to try.
size = 100
# Set the dimensionality of the distributions.
dims = 20

# Create mean and standard deviations vectors for the group A distributions.
locs_a = np.random.multivariate_normal(np.repeat(0, dims), np.diag(np.repeat(0.01, dims)), size=size)
scales_a = np.random.multivariate_normal(np.repeat(1, dims), np.diag(np.repeat(0.01, dims)), size=size) ** 2

# Create mean and standard deviations vectors for the group B distributions.
locs_b = np.random.multivariate_normal(np.repeat(0, dims), np.diag(np.repeat(0.01, dims)), size=size)
scales_b = np.random.multivariate_normal(np.repeat(1, dims), np.diag(np.repeat(0.01, dims)), size=size) ** 2

# Create instances of the MultivariateCoupledNormal classes with the parameters above and zero coupling.
mvn_a = MultivariateCoupledNormal(loc=locs_a, scale=scales_a, kappa=0)
mvn_b = MultivariateCoupledNormal(loc=locs_b, scale=scales_b, kappa=0)

# Get the approximate kl divergences.
approx_kl_divs = coupled_kl_divergence_norm(mvn_a, mvn_b, kappa=0.0, alpha=2.0, root=False, n=10000, seed=1).squeeze()

# Create an empty list to hold the true KL divergences.
true_divs = []
# Loop through all the distributions.
for i in range(size):
    # Add the true KL divergence between the i-th distribution in group A and the i-th distribution in group B.
    true_divs.append(kl_divergence_MVN(locs_a[i], locs_b[i], np.diag(scales_a[i]**2), np.diag(scales_b[i]**2)))
# Convert the list to a numpy array.
true_divs = np.array(true_divs)

# Calculate the average true and approximate KL divergences. 
avg_kl = np.mean(true_divs)
avg_approx_kl = np.mean(approx_kl_divs)
# Calculate the RMSE of the approximations.
rmse = np.sqrt(np.mean((true_divs - approx_kl_divs)**2))
# Calculate the MAPE.
mape = 100*np.mean(np.abs(true_divs - approx_kl_divs)/true_divs)

In [147]:
print(f"Avg. True KL Divergence: {round(avg_kl, 5)}")
print(f"Avg. Estimated KL Divergence: {round(avg_approx_kl, 5)}")
print(f"RMSE: {round(rmse, 5)}")
print(f"MAPE {round(mape, 3)}%")

Avg. True KL Divergence: 1.98172
Avg. Estimated KL Divergence: 1.97844
RMSE: 0.02549
MAPE 1.043%


C:\Users\jkcle\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
